In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinyCNN(nn.Module):
    def __init__(self, num_classes):
        super(TinyCNN, self).__init__()
        
        # --- Block 1 ---
        # Input: (Batch_Size, 3, 64, 64)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) 
        # Output after pool: (32, 32, 32)

        # --- Block 2 ---
        # Input: (32, 32, 32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        # Output after pool: (64, 16, 16)

        # --- Block 3 ---
        # Input: (64, 16, 16)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        # Output after pool: (128, 8, 8)

        # --- Classifier ---
        # The flatten size is: Channels * Height * Width
        # 128 channels * 8 pixels * 8 pixels = 8192
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Flatten: (Batch_Size, 128, 8, 8) -> (Batch_Size, 8192)
        x = x.view(x.size(0), -1)
        
        # Classifier
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [ ]:
# 1. Setup the model (e.g., if you are classifying cats vs dogs, num_classes=2)
model = TinyCNN(num_classes=2)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 2. Test with a dummy input to verify shapes match
# Batch size 8, 3 Channels, 64 Height, 64 Width
dummy_input = torch.randn(8, 3, 64, 64).to(device)
output = model(dummy_input)

print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}") # Should be [8, 2]